# openai stored completions

- https://platform.openai.com/docs/api-reference/chat/create (store completion request with store=true)
- https://platform.openai.com/docs/api-reference/chat/list (list stored completions)

## prepare sample chat data

In [8]:
import requests
from pathlib import Path

# https://huggingface.co/datasets/Samsung/samsum
samsum_dataset_url = "https://huggingface.co/datasets/Samsung/samsum/resolve/main/data/corpus.7z?download=true"
data_dir = Path("./data/samsum")
data_dir.mkdir(exist_ok=True)
data_file = data_dir / "corpus.7z"

with requests.get(samsum_dataset_url, stream=True) as response:
    response.raise_for_status()
    with open(data_file, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

In [ ]:
!pip install py7zr

In [10]:
import py7zr

with py7zr.SevenZipFile(data_file, mode="r") as z:
    z.extractall(data_dir)

In [18]:
import json

with open(data_dir / "train.json", encoding="utf8") as f:
    samsum_data = json.load(f)

## store chat completions with store=true

In [44]:
from openai import AzureOpenAI
import json

with open("openai_api.json") as f:
    creds = json.load(f)

client = AzureOpenAI(
    api_version=creds["api_version2"],
    azure_endpoint=creds["api_base2"],
    api_key=creds["api_key2"],
)

In [48]:
deployment = "gpt-4o-2024-08-06"

for i, row in enumerate(samsum_data[:20]):
    messages = [
        { "role": "system", "content": "You are a dialogue summarizer." },
        { "role": "user", "content": f"Summarize this dialogue:\n\n{row['dialogue']}"}
    ]

    response = client.chat.completions.with_raw_response.create(
        messages=messages,
        model=deployment,
        temperature=0.1,
        store=True,
        metadata={
            "source": "notebook"
        }
    )
    response_json = json.loads(response.content)
    print(i, ", Completion id:", response_json["id"], ", Request id:", response.headers.get("apim-request-id"))

## manage stored completions

### list completions - https://platform.openai.com/docs/api-reference/chat/list

In [63]:
completions = client.chat.completions.list(
  metadata={"source": "notebook"},
  limit=2,  # completions in one page
)

for page, completion in enumerate(completions):
    print(page, completion.id, completion.request_id)
    if page > 2:
        break

### get completion - https://platform.openai.com/docs/api-reference/chat/get

In [66]:
completion_id = "chatcmpl-B2u5LdHbuRqgYA1bY4Xe30iIZdwUZ"

completion = client.chat.completions.retrieve(completion_id)
print(completion.id, completion.request_id)

### get completion input messages - https://platform.openai.com/docs/api-reference/chat/getMessages

In [69]:
messages = client.chat.completions.messages.list(
    completion_id,
    limit=2,
)

for page, message in enumerate(messages):
    print(page, message.role, message.content)
    if page > 2:
        break

### update completion metadata - https://platform.openai.com/docs/api-reference/chat/update

In [72]:
completion = client.chat.completions.update(
  completion_id,
  metadata={
      "tag_to_add": "value",  # add new tag
      "source": "notebook2",  # update existing tag
      "tag_to_delete": None,  # delete existing tag
  }
)
print(completion.id, completion.metadata)

### delete completion - https://platform.openai.com/docs/api-reference/chat/delete

In [73]:
delete_response = client.chat.completions.delete(completion_id)
print(delete_response.id, delete_response.deleted)